In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
import os
import numpy as np


In [ ]:
# Read the array from disk
new_data = np.loadtxt('test.csv')

# Note that this returned a 2D array!
print(new_data.shape)

# However, going back to 3D is easy if we know the 
# original shape of the array
new_data = new_data.reshape((1000, 20, 4))
print(new_data[0][0])
# Just to check that they're the same...
# assert np.all(new_data == data)

In [ ]:
x_input = pd.read_csv('model_input.csv').to_numpy()

# x_input = np.loadtxt('model_input.csv')
# x_input = x_input.reshape((1000, 20, 4))
# print(x_input.shape)
# 3/4 training, 1/4 test
x_train = torch.tensor(x_input[:750], dtype=torch.float)
x_test = torch.tensor(x_input[750:], dtype=torch.float)

y_input = pd.read_csv('model_expected_output.csv').to_numpy()
# y_input = np.loadtxt('model_expected_output.csv')
# y_input = y_input.reshape(1000, 4)
# print(y_input.shape)

y_train = torch.tensor(y_input[:750], dtype=torch.float)
y_test = torch.tensor(y_input[750:], dtype=torch.float)

train_dataset = TensorDataset(x_train,y_train) # create your datset
train_dataloader = DataLoader(train_dataset) # create your dataloader

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset)

# print(test_dataloader.dataset[0][0].shape)
# print(test_dataloader.dataset[0][0])

# """
#   Standard Scale test and train data
#   Z - Score normalization
# """
# def scale_datasets(x_train, x_test):
#     standard_scaler = StandardScaler()
#     x_train_scaled = pd.DataFrame(
#         standard_scaler.fit_transform(x_train),
#         columns=x_train.columns
#     )
#     x_test_scaled = pd.DataFrame(
#         standard_scaler.transform(x_test),
#         columns = x_test.columns
#     )
#     return x_train_scaled, x_test_scaled
# x_train_scaled, x_test_scaled = scale_datasets(train_dataset, test_dataset)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        hidden_units1 = 160
        hidden_units2 = 480
        hidden_units3 = 256
        self.linear_relu_stack = nn.Sequential(
            # nn.Flatten(),
            # nn.Linear(80, 4)
            nn.Linear(80, 4),
            nn.ReLU(),
            nn.Dropout(0.5),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        # print(f"predicted: {pred}")
        # print(f"actual: {y}")
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print(f"loss: {loss:>7f}")

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
model = NeuralNetwork()
learning_rate = 0.001
batch_size = 64
epochs = 5
# Initialize the loss function
# loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss() # can change this to another loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    
print("Done!")